### Import libraries

In [3]:
import cv2
import csv

### Define paramerters and API key

In [4]:
PAT = '858d33cd787c47ccb0ab96e6728dbfd4'
USER_ID = 'leumas'
APP_ID = 'my-first-application'
MODEL_ID = 'ocr-scene-english-paddleocr'
MODEL_VERSION_ID = '40dbb2c9cde44a27af226782e7157006'
IMAGE_FILE_LOCATION = './image/input_half.png'


from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

with open(IMAGE_FILE_LOCATION, "rb") as f:
    file_bytes = f.read()

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        base64=file_bytes
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)


output = post_model_outputs_response.outputs[0]





### Importing original image to refer original size

In [8]:
image = cv2.imread(IMAGE_FILE_LOCATION)
image.shape

(2524, 2852, 3)

In [10]:
for box in post_model_outputs_response.outputs[0].data.regions:
        print("name",box.data.text.raw)
        print("top",int(box.region_info.bounding_box.top_row * image.shape[0]))
        print("left", int(box.region_info.bounding_box.left_col * image.shape[1]))
        print("bottom", int(box.region_info.bounding_box.bottom_row * image.shape[0]))
        print("right",int(box.region_info.bounding_box.right_col * image.shape[1]))
        print("")


name 2
top 788
left 1585
bottom 875
right 1652

name 2
top 993
left 1439
bottom 1096
right 1518

name 2
top 1064
left 631
bottom 1163
right 682

name 80
top 1175
left 1483
bottom 1242
right 1569

name 40
top 1313
left 729
bottom 1376
right 816

name 10
top 1313
left 2493
bottom 1384
right 2575

name :
top 1589
left 1443
bottom 1683
right 1510

name 2
top 1648
left 646
bottom 1715
right 698

name 0
top 1703
left 1968
bottom 1758
right 2015

name :
top 1920
left 1443
bottom 2015
right 1518

name 50
top 2169
left 682
bottom 2236
right 769

name 120
top 2173
left 1471
bottom 2236
right 1585

name 30
top 2173
left 2193
bottom 2236
right 2280



### Define function to get bounding box coordinates and write to solo file

In [63]:
# with open('ocr_bb.csv', 'w') as myfile:
#     for box in post_model_outputs_response.outputs[0].data.regions:
#         wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#         name = box.data.text.raw
#         top = int(box.region_info.bounding_box.top_row * image.shape[0])
#         left = int(box.region_info.bounding_box.left_col * image.shape[1])
#         bottom = int(box.region_info.bounding_box.bottom_row * image.shape[0])
#         right = int(box.region_info.bounding_box.right_col * image.shape[1])
# 
#         wr.writerow([name, top, left, bottom, right])

### Define function to get bounding box coordinates and write to common.csv file

In [64]:
with open('common.csv', 'a') as myfile:
    for box in post_model_outputs_response.outputs[0].data.regions:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        name = box.data.text.raw
        top = int(box.region_info.bounding_box.top_row * image.shape[0])
        left = int(box.region_info.bounding_box.left_col * image.shape[1])
        bottom = int(box.region_info.bounding_box.bottom_row * image.shape[0])
        right = int(box.region_info.bounding_box.right_col * image.shape[1])

        wr.writerow(["Dimension Text", name, -1, -1, left, top, right, bottom, -1])